In [ ]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df

# Select your features (columns)

In [ ]:
# Set features. This will also be used as your x values.
selected_features = df.drop("koi_disposition", axis=1)
feature_names = selected_features.columns

# Create a Train Test Split

Use `koi_disposition` for the y values

In [ ]:
# assign X(data) and y(target)
data = selected_features
target = df["koi_disposition"]
#y = df["koi_disposition"].values.reshape(-1, 1)
print(data.shape, target.shape)

In [ ]:
# split the data into training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [ ]:
X_train

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# create a random forest classifier to automatically calculate feature importance
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

In [ ]:
# sort the features by their importance
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

In [ ]:
print(X_train_scaled.shape, X_test_scaled.shape)

In [ ]:
# use tree-based estimator coupled with SelectFromModel meta-transformer...
# ...to compute impurity-based feature importances and discard irrelevant features 
from sklearn.feature_selection import SelectFromModel
sfmodel = SelectFromModel(rf, prefit=True)
X_train_scaled = sfmodel.transform(X_train_scaled)
X_test_scaled = sfmodel.transform(X_test_scaled)
print(X_train_scaled.shape, X_test_scaled.shape)

# Train the Model



In [ ]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model

In [ ]:
# fit(train) our model using the training data
model.fit(X_train_scaled, y_train)

In [ ]:
# validate the model using the test data
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model/estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {'random_state': [0, 42],
              'tol': [1, 100, 1000, 10000, 10000, 1000000, 10000000],
              'degree': [1, 2, 3],
              'kernel': ['linear', 'poly', 'rbf'],
              'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch estimator...
# this will take the model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

In [ ]:
# list the best parameters for this dataset, then list the best score
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
# make predictions with the hypertuned model
predictions = grid.predict(X_test_scaled)

In [ ]:
# calculate classification report
# precision = true positive / (true positive + false positive)
# recall = true positive / (true positive + false negative)
# f1 = 2*((precision * recall) / (precision + recall)): best at 1, worst at 0
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

# Save the Model

In [ ]:
import joblib
filename = 'exoplanets_SVC.sav'
joblib.dump(grid, filename)